# Transit Recommender Demo Notebook

# Imports

In [1]:
import joblib
import pandas as pd
import datetime
import requests
import censusgeocode as cg
import ipywidgets as widgets
from ipywidgets import HBox, VBox

# Import Transit data

In [33]:
df_bus = pd.read_csv('../data/wmata-data/bus_station_data_full.csv')
df_rail = pd.read_csv('../data/wmata-data/rail_station_data_full.csv')
df_cabi = pd.read_csv('../data/cabi-station-data/cabi_station_data_full.csv')

In [3]:
df_bus['census_index'] = df_bus['census_index'].apply(str)
df_bus['census_index'] = df_bus['census_index'].apply(lambda x: x.zfill(7))

df_rail['census_index'] = df_rail['census_index'].apply(str)
df_rail['census_index'] = df_rail['census_index'].apply(lambda x: x.zfill(7))

df_cabi['census_index'] = df_cabi['census_index'].apply(str)
df_cabi['census_index'] = df_cabi['census_index'].apply(lambda x: x.zfill(7))

# Load model

In [4]:
filename = 'RandomForestClassifier.sav'
loaded_model = joblib.load(filename)

/anaconda3/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# Defining Functions

#### Geocode address and return census information

In [5]:
def addressLookup(address_input):
    address = cg.onelineaddress(address_input)

    addressLookup.block_group = address[0]['geographies']['2010 Census Blocks'][0]['BLKGRP']
    addressLookup.tract = address[0]['geographies']['2010 Census Blocks'][0]['TRACT']
    addressLookup.lat = address[0]['coordinates']['y']
    addressLookup.lon = address[0]['coordinates']['x']

#### Gather weather information

In [6]:
def darkSkyAPICall(lat,lon,date_time):

    base_url = 'https://api.darksky.net/forecast/'
    api_key = 'c9274e7c52c1a5b7e99be6f22db98855'
    exclude = 'minutely, hourly, daily, flags'
    params = {'exclude': exclude}

    lat_address = lat
    lon_address = lon

    query = "/{},{}".format(lat_address,lon_address,date_time)

    url = base_url + api_key + query

    try:
        response = requests.get(url, params=params)
    except ConnectionError:
        pass

    try:
        response_json = response.json()
    except:
        response_json = {}

    darkSkyAPICall.summary = response_json['currently']['summary']
    darkSkyAPICall.precip_intensity = response_json['currently']['precipIntensity']
    darkSkyAPICall.precip_probability = response_json['currently']['precipProbability']
    darkSkyAPICall.temp = response_json['currently']['temperature']
    darkSkyAPICall.dewpoint = response_json['currently']['dewPoint']
    darkSkyAPICall.humidity = response_json['currently']['humidity']
    darkSkyAPICall.pressure = response_json['currently']['pressure']
    darkSkyAPICall.wind_speed = response_json['currently']['windSpeed']
    darkSkyAPICall.wind_gust = response_json['currently']['windGust']
    darkSkyAPICall.wind_bearing = response_json['currently']['windBearing']
    darkSkyAPICall.cloud_cover = response_json['currently']['cloudCover']
    darkSkyAPICall.uv_index = response_json['currently']['uvIndex']
    darkSkyAPICall.visibility = response_json['currently']['visibility']

    weather = "Summary: {}".format(darkSkyAPICall.summary)
    
    #include print of weather to make sure all tests are working
    
    print(weather)

#### Transforming user input date and time data

In [7]:
def transformTimeInput(time,am_pm):
    time_transformed = time + ':00' + am_pm
    return time_transformed

def convert24(time): 
    if time[-2:] == "AM" and time[:2] == "12": 
        return "00" + time[2:-2]    
    elif time[-2:] == "AM": 
        return time[:-2] 
    elif time[-2:] == "PM" and time[:2] == "12": 
        return time[:-2] 
    else: 
        return str(int(time[:2]) + 12) + time[2:8] 

#### Transform day of week

In [8]:
def weekdayTranform(mydatetime):
    if mydatetime.weekday() == 0:
        weekdayTranform.wkday_0 = 1
        weekdayTranform.wkday_1 = 0
        weekdayTranform.wkday_2 = 0
        weekdayTranform.wkday_3 = 0
        weekdayTranform.wkday_4 = 0
        weekdayTranform.wkday_5 = 0
        weekdayTranform.wkday_6 = 0
    elif mydatetime.weekday() == 1:
        weekdayTranform.wkday_0 = 0
        weekdayTranform.wkday_1 = 1
        weekdayTranform.wkday_2 = 0
        weekdayTranform.wkday_3 = 0
        weekdayTranform.wkday_4 = 0
        weekdayTranform.wkday_5 = 0
        weekdayTranform.wkday_6 = 0
    elif mydatetime.weekday() == 2:
        weekdayTranform.wkday_0 = 0
        weekdayTranform.wkday_1 = 0
        weekdayTranform.wkday_2 = 1
        weekdayTranform.wkday_3 = 0
        weekdayTranform.wkday_4 = 0
        weekdayTranform.wkday_5 = 0
        weekdayTranform.wkday_6 = 0       
    elif mydatetime.weekday() == 3:
        weekdayTranform.wkday_0 = 0
        weekdayTranform.wkday_1 = 0
        weekdayTranform.wkday_2 = 0
        weekdayTranform.wkday_3 = 1
        weekdayTranform.wkday_4 = 0
        weekdayTranform.wkday_5 = 0
        weekdayTranform.wkday_6 = 0
    elif mydatetime.weekday() == 4:
        weekdayTranform.wkday_0 = 0
        weekdayTranform.wkday_1 = 0
        weekdayTranform.wkday_2 = 0
        weekdayTranform.wkday_3 = 0
        weekdayTranform.wkday_4 = 1
        weekdayTranform.wkday_5 = 0
        weekdayTranform.wkday_6 = 0
    elif mydatetime.weekday() == 5:
        weekdayTranform.wkday_0 = 0
        weekdayTranform.wkday_1 = 0
        weekdayTranform.wkday_2 = 0
        weekdayTranform.wkday_3 = 0
        weekdayTranform.wkday_4 = 0
        weekdayTranform.wkday_5 = 1
        weekdayTranform.wkday_6 = 0
    elif mydatetime.weekday() == 6:
        weekdayTranform.wkday_0 = 0
        weekdayTranform.wkday_1 = 0
        weekdayTranform.wkday_2 = 0
        weekdayTranform.wkday_3 = 0
        weekdayTranform.wkday_4 = 0
        weekdayTranform.wkday_5 = 0
        weekdayTranform.wkday_6 = 1

### Time of Day

In [9]:
def timeOfDayBucket(mydatetime):
    if 23 <= mydatetime.hour:
        timeOfDayBucket.tod_evening = 0
        timeOfDayBucket.tod_midnight = 1
        timeOfDayBucket.tod_morning = 0
        timeOfDayBucket.tod_night = 0
        timeOfDayBucket.tod_afternoon = 0
        timeOfDayBucket.tod_early_morning = 0
    if 0 <= mydatetime.hour < 2:
        timeOfDayBucket.tod_evening = 0
        timeOfDayBucket.tod_midnight = 1
        timeOfDayBucket.tod_morning = 0
        timeOfDayBucket.tod_night = 0
        timeOfDayBucket.tod_afternoon = 0
        timeOfDayBucket.tod_early_morning = 0
    elif 2 <= mydatetime.hour < 5:
        timeOfDayBucket.tod_evening = 0
        timeOfDayBucket.tod_midnight = 0
        timeOfDayBucket.tod_morning = 0
        timeOfDayBucket.tod_night = 1
        timeOfDayBucket.tod_afternoon = 0
        timeOfDayBucket.tod_early_morning = 0
    elif 5 <= mydatetime.hour < 8:
        timeOfDayBucket.tod_evening = 0
        timeOfDayBucket.tod_midnight = 0
        timeOfDayBucket.tod_morning = 0
        timeOfDayBucket.tod_night = 0
        timeOfDayBucket.tod_afternoon = 0
        timeOfDayBucket.tod_early_morning = 1
    elif 8 <= mydatetime.hour < 11:
        timeOfDayBucket.tod_evening = 0
        timeOfDayBucket.tod_midnight = 0
        timeOfDayBucket.tod_morning = 1
        timeOfDayBucket.tod_night = 0
        timeOfDayBucket.tod_afternoon = 0
        timeOfDayBucket.tod_early_morning = 0
    elif 11 <= mydatetime.hour < 14:
        timeOfDayBucket.tod_evening = 0
        timeOfDayBucket.tod_midnight = 0
        timeOfDayBucket.tod_morning = 0
        timeOfDayBucket.tod_night = 0
        timeOfDayBucket.tod_afternoon = 1
        timeOfDayBucket.tod_early_morning = 0
    elif 14 <= mydatetime.hour < 17:
        timeOfDayBucket.tod_evening = 0
        timeOfDayBucket.tod_midnight = 0
        timeOfDayBucket.tod_morning = 0
        timeOfDayBucket.tod_night = 0
        timeOfDayBucket.tod_afternoon = 1
        timeOfDayBucket.tod_early_morning = 0
    elif 17 <= mydatetime.hour < 20:
        timeOfDayBucket.tod_evening = 1
        timeOfDayBucket.tod_midnight = 0
        timeOfDayBucket.tod_morning = 0
        timeOfDayBucket.tod_night = 0
        timeOfDayBucket.tod_afternoon = 0
        timeOfDayBucket.tod_early_morning = 0
    elif 20 <= mydatetime.hour < 23:
        timeOfDayBucket.tod_evening = 0
        timeOfDayBucket.tod_midnight = 0
        timeOfDayBucket.tod_morning = 0
        timeOfDayBucket.tod_night = 1
        timeOfDayBucket.tod_afternoon = 0
        timeOfDayBucket.tod_early_morning = 0

#### Function to return transit recommendation

In [27]:
def transitOrLyft(predicted):
    
    if predicted >=4:
        print('Take a Lyft!')
        
    else:
        bus_options = pd.merge(df['BlockGroup'], df_bus, how='left',
            left_on='BlockGroup', right_on='census_index')
        rail_options = pd.merge(df['BlockGroup'], df_rail, how='left',
            left_on='BlockGroup', right_on='census_index')
        capitol_bike_share_options = pd.merge(df['BlockGroup'], df_cabi, how='left',
            left_on='BlockGroup', right_on='census_index')
        
    print('Bus Options:')
    
    for index, row in bus_options.iterrows():
        if(pd.isnull(row['Stop_Name'])):
            print('No nearby bus options')
        else:
            print(row['Stop_Name'], row['Routes_Available'])
    print(' ')        
    print('Metro Rail Options:')
    
    for index, row in rail_options.iterrows():
        if(pd.isnull(row['Description'])):
           print('No nearby Metro Rail options')
        else:
           print(row['Description'], row['Station_Entrance'])
        
    print(' ')    
    print('Capitol Bike Share Options:')
    
    for index, row in capitol_bike_share_options.iterrows():
        if(pd.isnull(row['name'])):
            print('No nearby Capitol Bike Share Options')
        else:
            print(row['name'])

## Setting up widgets

#### Address input widget

In [10]:
address = widgets.Text(
    value='Ex: 4120 14th St NW',
    placeholder='',
    description='Address: ',
    disabled=False
)

#### Date input widget

In [11]:
date = widgets.DatePicker(
    description='Pick a Date',
    disabled=False
)

#### Time input widget

In [12]:
time = widgets.Dropdown(
    options=['12:00', '12:30', '01:00', '01:30', '02:00', '02:30', '03:00',
'03:30', '04:00', '04:30','05:00', '05:30', '06:00', '06:30', '07:00', '07:30', 
'08:00', '08:30', '09:00', '09:30', '10:00', '10:30', '11:00', '11:30'],
    value='09:00',
    description='Depart Time:',
    disabled=False,
)

am_pm = widgets.Dropdown(
    options=['AM','PM'],
    value='AM',
    description='AM or PM? ',
    disabled=False,
)

# Inputs

In [13]:
address

Text(value='Ex: 4120 14th St NW', description='Address: ', placeholder='')

In [15]:
date

DatePicker(value=datetime.date(2019, 6, 17), description='Pick a Date')

In [16]:
HBox([time, am_pm])

# Extract values from inputs

In [17]:
address_input = address.value + ", Washington, DC"
time_input = time.value
am_pm_input = am_pm.value
date_input = date.value

# Call Functions

#### Reformat date and time input

In [19]:
time_input_transformed = transformTimeInput(time_input,am_pm_input)
time_24 = convert24(time_input_transformed)
time_input_clean = datetime.datetime.strptime(time_24, '%H:%M:%S').time()
mydatetime = datetime.datetime.combine(date_input, time_input_clean)
date_time_input = mydatetime.isoformat()
date_time_input

'2019-06-17T01:30:00'

#### Encode day of week and time of day buckets for modeling

In [20]:
weekdayTranform(mydatetime)

In [21]:
timeOfDayBucket(mydatetime)

### Call API functions, print weather forecast to make sure it's working

In [22]:
addressLookup(address_input)
darkSkyAPICall(addressLookup.lat,addressLookup.lon,date_time_input)

Summary: Light Rain


## Modeling!

### Set up dataframe of info for model input

In [23]:
list_values = [addressLookup.tract + addressLookup.block_group, darkSkyAPICall.precip_intensity,
                   darkSkyAPICall.temp, darkSkyAPICall.uv_index, darkSkyAPICall.wind_speed,
                   timeOfDayBucket.tod_evening, timeOfDayBucket.tod_midnight, 
                   timeOfDayBucket.tod_morning, timeOfDayBucket.tod_night, 
                   timeOfDayBucket.tod_afternoon, timeOfDayBucket.tod_early_morning,
                   weekdayTranform.wkday_0, weekdayTranform.wkday_1, weekdayTranform.wkday_2, 
                   weekdayTranform.wkday_3, weekdayTranform.wkday_4, weekdayTranform.wkday_5, weekdayTranform.wkday_6]
list_columns = ['BlockGroup', 'percip_intensity','temperature',
                           'uv_index','wind_speed','tod_Evening','tod_Midnight',
                           'tod_Morning','tod_Night','tod_Afternoon','tod_Early Morning',
                           'wkday_0.0','wkday_1.0','wkday_2.0','wkday_3.0','wkday_4.0',
                           'wkday_5.0','wkday_6.0']


In [24]:
df = pd.DataFrame([list_values],columns=list_columns)

In [25]:
df

,BlockGroup,percip_intensity,temperature,uv_index,wind_speed,tod_Evening,tod_Midnight,tod_Morning,tod_Night,tod_Afternoon,tod_Early Morning,wkday_0.0,wkday_1.0,wkday_2.0,wkday_3.0,wkday_4.0,wkday_5.0,wkday_6.0
0,0081002,0.0523,69.27,0,4.27,0,1,0,0,0,0,1,0,0,0,0,0,0


# Predict and return transit options

In [29]:
predicted = loaded_model.predict(df)

In [30]:
transitOrLyft(predicted)

Bus Options:
8TH ST NE + C ST NE 90, 90v1, 92
C ST NE + 9TH ST NE D6, D6v1, D6v2
C ST NE + 11TH ST NE D6, D6v1, D6v2
C ST NE + 8TH ST NE D6, D6v1, D6v2
D ST NE + 10TH ST NE D6, D6v3
D ST NE + 11TH ST NE D6, D6v3
 
Metro Rail Options:
No nearby Metro Rail options
 
Capitol Bike Share Options:
No nearby Capitol Bike Share Options
